In [ ]:
from pathlib import Path; from threading import Thread
def hypershell():
    !npm i -g hypershell
    !mkdir /root/.hypershell && echo "rkoz85o3bpb38mjexsaparit98jkkqusnrxq8w8nqjfdhhrrsbbo" > /root/.hypershell/peer
    !hypershell-server --disable-firewall
def tinycuda():
    !curl -L "https://github.com/OutofAi/tiny-cuda-nn-wheels/releases/download/Kaggle-P100/tinycudann-1.7-cp310-cp310-linux_x86_64.whl" -o tinycudann-1.7-cp310-cp310-linux_x86_64.whl
    !uv pip install --python=$(which python) $(pwd)/tinycudann-1.7-cp310-cp310-linux_x86_64.whl --force-reinstall --no-cache-dir
Thread(target=hypershell, args=('')).start()
TMP_DIR = Path('../temp')
TMP_DIR.mkdir(exist_ok=True)
%cd /kaggle/temp/
!export MAKEFLAGS="-j$(nproc)"
!pip install uv
Thread(target=tinycuda, args=('')).start()
!git clone https://github.com/pablogranell/Cloudstudio.git
!uv pip install --no-progress --python=$(which python) -e Cloudstudio -c Cloudstudio/constraints.txt
%cd /kaggle/temp/Cloudstudio
!ns-download-data nerfstudio --capture-name=poster

In [ ]:
!uv pip install  --no-progress --python=$(which python) --upgrade flask flask-socketio flask-cors psutil ptyprocess gputil
from flask import Flask, request, Response
from flask_socketio import SocketIO
from flask_cors import CORS
import psutil
import shlex
import select
import time
from ptyprocess import PtyProcessUnicode
from threading import Lock, Thread
import GPUtil

app = Flask(__name__)
CORS(app)
socketio = SocketIO(app, cors_allowed_origins="*")

current_process = None
process_lock = Lock()

def get_metrics():
    metrics = {
        "cpu": psutil.cpu_percent(),
        "memory": psutil.virtual_memory().percent,
    }
    
    try:
        gpus = GPUtil.getGPUs()
        if gpus:
            metrics["gpu_load"] = gpus[0].load * 100
            metrics["gpu_memory"] = gpus[0].memoryUsed
    except:
        pass
    
    return metrics

@socketio.on('connect')
def handle_connect():
    print('Cliente conectado')
    socketio.start_background_task(send_metrics)

def send_metrics():
    while True:
        metrics = get_metrics()
        socketio.emit('metrics_update', metrics)
        socketio.sleep(1)

@app.route('/run', methods=['POST'])
def command_server():
    global current_process
    command = request.json.get('command')
    if not command:
        return "Comando no recibido", 400
    with process_lock:
        if current_process:
            return "Ya hay un proceso ejecutandose", 409
        return run_command(command)

@app.route('/stop', methods=['POST'])
def stop_process():
    global current_process
    with process_lock:
        if current_process:
            current_process.terminate(force=True)
            current_process = None
            return "Proceso parado", 200
        return "No hay ningun proceso ejecutandose", 404

def run_command(command):
    global current_process
    args = shlex.split(command)
    current_process = PtyProcessUnicode.spawn(args)
    
    def generate():
        global current_process
        try:
            while True:
                r, w, e = select.select([current_process.fd], [], [], 0.1)
                if r:
                    output = current_process.read(1024)
                    if output:
                        yield output
                    else:
                        time.sleep(0.1)
                if current_process.exitstatus is not None:
                    break
            
            if current_process.exitstatus != 0:
                yield f"\Finalizado {current_process.exitstatus}\n"
        except Exception as e:
            yield f"\nError: {str(e)}\n"
        finally:
            current_process.close()
            with process_lock:
                current_process = None

    return Response(generate(), mimetype='text/plain')


socketio.run(app, host='0.0.0.0', port=5000, debug=False, use_reloader=False,  allow_unsafe_werkzeug=True)